In [8]:
import pandas as pd
import numpy as np

def find_habitable_planets(file_path, output_file="habitable_planets.csv"):
    # Load data and skip comment lines # 
    with open(file_path, "r") as file:
        lines= file.readlines()
    for i, line in enumerate(lines):
        if not line.startswith("#"):
            header_line= i
            break
    data= pd.read_csv(file_path, skiprows=header_line)

    # Drop rows missing data
    columns_needed= ["pl_name", "hostname", "pl_rade", "pl_orbper", "st_teff", "st_spectype", "st_lum"]
    data = data.dropna(subset=columns_needed).copy()

    # Calculate orbital distance via Kepler’s Third Law
    data["orbital_years"]= data["pl_orbper"]/365
    data["orbital_distance"]= data["orbital_years"]**(2 / 3)

    # Convert log luminosity to solar units
    data["luminosity"] = 10 ** data["st_lum"]

    # Calculate inner and outer habitable zone boundaries
    data["hz_inner"]= np.sqrt(data["luminosity"] / 1.1)
    data["hz_outer"]= np.sqrt(data["luminosity"] / 0.53)

    # Filter by habitability: radius inner/outer parameters note I change this to .5-2.0 orginal was .5-1.5
    #star type note also Included M type stars and temp
    #orbit distance note I also included a +-5% margin of error for habitiability zones
    # and +-15% HZ for near habitability planets
    size_ok = (data["pl_rade"] >= 0.5) & (data["pl_rade"] <= 2.0)
    temp_ok = (data["st_teff"] >= 4800) & (data["st_teff"] <= 6300)
    type_ok = data["st_spectype"].str.startswith(("F", "G", "K", "M"))
    dist = data["orbital_distance"]
    in_hz = (dist >= data["hz_inner"] * 0.95) & (dist <= data["hz_outer"] * 1.05)
    near_hz = ((dist >= data["hz_inner"] * 0.85) & (dist <= data["hz_outer"] * 1.15)) & ~in_hz

    hab = data[size_ok & temp_ok & type_ok & in_hz].copy()
    near = data[size_ok & temp_ok & type_ok & near_hz].copy()

    # Print habitable planet results
    print("Potentially Habitable Planets\n")
    for _, row in hab.iterrows():
        print(f"- {row['pl_name']} (Host:{row['hostname']})")
        print(f"  Radius:{row['pl_rade']:.2f} Earth radii")
        print(f"  Distance:{row['orbital_distance']:.2f} AU")
        print(f"  Temp:{row['st_teff']:.0f} K | Type: {row['st_spectype']}")
        print(f"  HZ Range:{row['hz_inner']:.2f}–{row['hz_outer']:.2f} AU\n")

    #print near-habitable planets
    print("Near-Habitable Planets (just outside HZ)\n")
    for _, row in near.iterrows():
        print(f"- {row['pl_name']} (Host: {row['hostname']})")
        print(f"  Radius: {row['pl_rade']:.2f} Earth radii")
        print(f"  Distance: {row['orbital_distance']:.2f} AU")
        print(f"  Temp: {row['st_teff']:.0f} K | Type: {row['st_spectype']}")
        print(f"  HZ Range: {row['hz_inner']:.2f} – {row['hz_outer']:.2f} AU\n")

    # Save habitable planets to a CSV file
    hab.to_csv(output_file, index=False)
    print(f"Saved {len(hab)} habitable planets to '{output_file}'.")

find_habitable_planets("project_data_NASA.csv")


Potentially Habitable Planets

- Kepler-452 b (Host:Kepler-452)
  Radius:1.63 Earth radii
  Distance:1.04 AU
  Temp:5757 K | Type: G2
  HZ Range:1.05–1.51 AU

- Kepler-62 e (Host:Kepler-62)
  Radius:1.61 Earth radii
  Distance:0.48 AU
  Temp:4925 K | Type: K2 V
  HZ Range:0.44–0.63 AU

Near-Habitable Planets (just outside HZ)

- Kepler-69 c (Host: Kepler-69)
  Radius: 1.71 Earth radii
  Distance: 0.76 AU
  Temp: 5638 K | Type: G4 V
  HZ Range: 0.85 – 1.23 AU

- tau Cet e (Host: tau Cet)
  Radius: 1.81 Earth radii
  Distance: 0.58 AU
  Temp: 5310 K | Type: G8.5V
  HZ Range: 0.67 – 0.97 AU

Saved 2 habitable planets to 'habitable_planets.csv'.
